# B. Runner and Customer Experience

<span style="font-size: 14px;"><b>1. How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)</b></span>

In [3]:
SELECT
	EXTRACT(week FROM registration_date) as week,
	COUNT(runner_id) as runners_count
FROM runners
GROUP BY week

(3 row(s) affected)

Total execution time: 00:00:00.003

week,runners_count
53.0,2
1.0,1
2.0,1


Here <span style="color: #09885a;">53</span> week means <span style="color: #09885a;">0</span> week

<span style="font-size: 14px;"><b>2.What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?</b></span>

- **time for pickup the orders is order\_time - pickup\_time**
- **where pickup\_time is not null which means the order had cancelled**

In [4]:
with cte as(
	SELECT 
		DISTINCT(c.order_id),
		c.order_time,
		r.runner_id,
		r.pickup_time,
		((DATE_PART('day', r.pickup_time::timestamp - c.order_time::timestamp) * 24 + 
               DATE_PART('hour', r.pickup_time::timestamp - c.order_time::timestamp)) * 60 +
               DATE_PART('minute', r.pickup_time::timestamp - c.order_time::timestamp)) as time_to_arrive_pizza_runner_HQ
	FROM new_customer_orders c
	JOIN new_runner_orders r
	ON c.order_id = r.order_id
	WHERE r.pickup_time is not null)
	SELECT
		runner_id,
		AVG(time_to_arrive_pizza_runner_HQ) as avg_time_to_arrive_pizza_runner_HQ
	FROM cte
	GROUP BY runner_id
	ORDER BY runner_id

(3 row(s) affected)

Total execution time: 00:00:00.004

runner_id,avg_time_to_arrive_pizza_runner_hq
1,14.0
2,19.666666666666668
3,10.0


<span style="font-size: 14px;"><b>3. Is there any relationship between the number of pizzas and how long the order takes to prepare?</b></span>

In [5]:
with cte as(
	SELECT
		*
	FROM (
		SELECT
	  		c.order_id,
			COUNT(c.pizza_id) as pizza_count
		FROM new_customer_orders c
		JOIN new_runner_orders r
			ON c.order_id = r.order_id
			WHERE r.pickup_time is not null
			GROUP BY c.order_id) as temp1
	JOIN (
		SELECT 
			DISTINCT(c.order_id),
			((DATE_PART('day', r.pickup_time::timestamp - c.order_time::timestamp) * 24 + 
               DATE_PART('hour', r.pickup_time::timestamp - c.order_time::timestamp)) * 60 +
               DATE_PART('minute', r.pickup_time::timestamp - c.order_time::timestamp)) as order_prepare_time
		FROM new_customer_orders c
		JOIN new_runner_orders r
			ON c.order_id = r.order_id
			WHERE r.pickup_time is not null) as temp2
	ON temp1.order_id = temp2.order_id
	ORDER BY temp1.order_id)
SELECT 
	pizza_count,
	avg(order_prepare_time)
FROM cte
GROUP BY pizza_count
ORDER BY pizza_count

(3 row(s) affected)

Total execution time: 00:00:00.003

pizza_count,avg
1,12.0
2,18.0
3,29.0


<span style="font-size: 14px;"><b>Observation:</b></span>

- pizza count increseas time to prepare is also incresead

<span style="font-size: 14px;"><b>4.What was the average distance travelled for each customer?</b></span>

In [6]:
with cte as(
	SELECT
		c.order_id,
		c.customer_id,
		r.distance
	FROM new_customer_orders c
	JOIN new_runner_orders r
		ON c.order_id = r.order_id
		WHERE r.distance is not null)
		SELECT 
			customer_id,
			ROUND(AVG(distance),2) as avg_distance
		FROM cte
		GROUP BY customer_id
		ORDER BY customer_id

(5 row(s) affected)

Total execution time: 00:00:00.003

customer_id,avg_distance
101,20.00
102,16.73
103,23.40
104,10.00
105,25.00


<span style="font-size: 14px;"><b>5. What was the difference between the longest and shortest delivery times for all orders?</b></span>

In [7]:
SELECT
	MAX(distance) as max_distance,
	MIN(distance) as min_distance,
	MAX(distance)-MIN(distance) as diff_btw_longest_shortest
FROM new_runner_orders r
	where distance is not null

(1 row(s) affected)

Total execution time: 00:00:00.002

max_distance,min_distance,diff_btw_longest_shortest
25,10,15


<span style="font-size: 14px;"><b>6. What was the average speed for each runner for each delivery and do you notice any trend for these values?</b></span>

In [8]:
WITH cte as(
SELECT 
	runner_id,
	distance,
	duration,
	ROUND((distance/duration),2) as speed_per_minute
FROM new_runner_orders 
where distance is not null)
SELECT 
	runner_id,
	ROUND(avg(speed_per_minute),2) avg_speed
FROM cte
GROUP BY runner_id
ORDER BY runner_id

(3 row(s) affected)

Total execution time: 00:00:00.002

runner_id,avg_speed
1,0.76
2,1.05
3,0.67


<span style="font-size: 14px;"><b>7. What is the successful delivery percentage for each runner?</b></span>

In [9]:
with cte as(
SELECT 
	runner_id,
	COUNT(*) as total_orders,
	COUNT(
		CASE
			WHEN cancellation is null THEN 1
		END) as delivered_orders,
		COUNT(
		CASE
			WHEN cancellation is not null THEN 1
		END) as cancelled_orders
FROM new_runner_orders
GROUP BY runner_id
ORDER BY runner_id)
SELECT
	*,
	ROUND((delivered_orders::decimal/total_orders::decimal)*100,2)
FROM cte

(3 row(s) affected)

Total execution time: 00:00:00.002

runner_id,total_orders,delivered_orders,cancelled_orders,round
1,4,4,0,100.00
2,4,3,1,75.00
3,2,1,1,50.00
